<a href="https://colab.research.google.com/github/koheiKESE/ai-seminer/blob/main/05_detectron2%E3%81%AB%E3%82%88%E3%82%8Bballoon%E3%81%AE%E8%AA%8D%E8%AD%98_%E5%AD%A6%E7%BF%92%E3%81%A8%E6%8E%A8%E8%AB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. このノートブックを実行する前すること

1. ランタイムをハードウェアアクセラレータをGPUに指定
  - ランタイム -> ランタイムのタイプを変更 -> ハードウェアアクセラレータをGPUに設定


2. balloonデータセットをGoogle Drive直下にコピー

3. このノートブックをGoogle Driveに保存 ( +コード＋テキストの右隣に表示されている「ドライブにコピー」ボタンをクリック

# 1. 初期設定

  1. Google Driveをマウント
    - pip installで指定した保存先 ('/content/drive/MyDrive/Colab Notebooks/my-.modules')のパスを通す

  2. detectron2とlabelme2cocoをインストール (**注意** : 90分/12時間ルールによりインストールしたものはリセットされます)

  3. colabの環境とdetectron2の環境間のバージョンのコンフリクトを解消するため、ランタイムを再起動 

In [ ]:
# Google Driveを /content/drive以下にマウント
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Pythonにmy-modulesフォルダのパスを登録
#import sys
#sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules/')

In [ ]:
# detectron2
# 所要時間 30秒程度
!pip install pyyaml==5.1
!pip install 'git+https://github.com/facebookresearch/detectron2.git' --upgrade

In [ ]:
# labelme2cocoのインストール
# labelme形式で保存されたjsonファイルをcoco形式に変換する
!pip install labelme2coco --upgrade

# インストール完了後、ランタイムを再起動

## 表示内容
WARNING: The following packages were previously imported in this runtime:
  [certifi]
You must restart the runtime in order to use newly installed versions.


## 解決方法 (どれかでＯＫ)
1. セルに表示される「RESTART RUNTIME」ボタンをクリック
2. メニューバーのランタイム -> ランタイムを再起動をクリック

# 正しくインストールできているか、モジュールをインポートして確認 (エラーが表示されなければOK)

In [ ]:
# detectron2のインポートとバージョン確認
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [ ]:
# labelme2cocoのインポート
import labelme2coco

In [ ]:
# 本プログラムで使用するライブラリのインポート
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer

from detectron2.data.datasets import register_coco_instances

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
import os
import glob

# バルーンデータセットが、本ノートブックから読み込めるかを確認

In [ ]:
# 画像が読み込めるかを確認
import cv2
import matplotlib.pyplot as plt
img = cv2.imread('/content/drive/MyDrive/balloon_labelme既アノテーション/train/10464445726_6f1e3bbe6a_k.jpg')
plt.imshow(img)
plt.show()

In [ ]:
!ls '/content/drive/MyDrive/balloon_labelme既アノテーション/train' 

In [ ]:
# 入力フォルダを指定
import_dir = '/content/drive/MyDrive/balloon_labelme既アノテーション/train'
# 出力フォルダを指定
export_dir = '/content/drive/MyDrive/balloon_labelme既アノテーション/train_labelme2coco_ano'

# set train split rate
# train_split_rate = 0.75

# convert labelme annotations to coco
labelme2coco.convert(import_dir, export_dir)

# 1. アノテーション結果の可視化

In [ ]:
# データセット名の指定 (識別用なので、名前はなんでもOK)
train_dataset_name = "balloon_train2"

# アノテーションデータの保存先
## balloon_labelme既アノテーション以下は、自分のフォルダ名に適宜書き換える
train_json_path = "/content/drive/MyDrive/balloon_labelme既アノテーション/train_labelme2coco_ano/dataset.json"

# 学習データのフォルダのパス
## balloon_labelme既アノテーション以下は、自分のフォルダ名に適宜書き換える
train_path = "/content/drive/MyDrive/balloon_labelme既アノテーション/train"

# 推論データのフォルダのパス
## balloon_labelme既アノテーション以下は、自分のフォルダ名に適宜書き換える
test_path = "/content/drive/MyDrive/balloon_labelme既アノテーション/val"

In [ ]:
# 訓練データセットをdetectron2に登録
# 引数1:データセットの名前(なんでもOK)、引数2:jsonファイル、引数3:画像が入っているフォルダ
register_coco_instances(train_dataset_name, {}, train_json_path, train_path)

In [ ]:
train_metadata = MetadataCatalog.get(train_dataset_name)
train_dicts = DatasetCatalog.get(train_dataset_name)

In [ ]:
# trainデータ数を表示
print(f"num_train_images : {len(train_dicts)}")

In [ ]:
# train_metadataを表示 (訓練データセットをdetectron2に登録した結果)
print(train_metadata)

In [ ]:
# train_dictsデータを表示 (アノテーション結果の座標等を表示)
print(train_dicts)

In [ ]:
# ランダムに画像+アノテーションを表示
import random

def plot_teacher_data(train_dicts, train_metadata, num=3):
    for d in random.sample(train_dicts, num):
        print(d["file_name"])
        img = cv2.imread(d["file_name"])
        visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
        vis = visualizer.draw_dataset_dict(d)
        cv2_imshow(vis.get_image()[:, :, ::-1])

  
plot_teacher_data(train_dicts, train_metadata, num=3)

# 2. detectron2で学習を実行
- 学習時間は、62枚で2分ぐらい
- 学習済みモデルはmodel_zooから任意のものを使用可能
- 主な変更は、NUM_CLASSES = 1 とすること。
- 学習が完了すると、学習済みモデルとしてmodel_final.pthが出力


In [ ]:
# 学習時のコンフィグ(設定)を記述

## コンフィグを初期化
cfg = get_cfg()

# モデルのロード (今回はResNet50+FPN_3x)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# 訓練データセットの登録
cfg.DATASETS.TRAIN = (train_dataset_name)

# テストデータセットの登録 (今回は空欄)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset

# 計算に使用するスレッド数の設定 (今回は2(=デフォルト値)を設定)
cfg.DATALOADER.NUM_WORKERS = 2

# ロードした学習済みモデルをモデルの重みの初期値に設定
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# バッチサイズ (今回は2を設定)
cfg.SOLVER.IMS_PER_BATCH = 2

# 学習率 (default値)
cfg.SOLVER.BASE_LR = 0.02

# MAXエポック数 (default値)
cfg.SOLVER.MAX_ITER = (
    300
)  # 300 iterations seems good enough, but you can certainly train longer

# ROIのバッチサイズ (default値)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset

# クラス数の登録 (今回は、balloonのみの検出なので1を設定)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 

cfg.OUTPUT_DIR = "/content/drive/MyDrive/output"

In [ ]:
# 学習実行

## outputフォルダを作成し、そこに学習結果を保存する。
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
## 上で設定した初期設定をtrainerにロード
trainer = DefaultTrainer(cfg)
##  最終チェックポイントかMODEL.WEIGHTSを読み込む (今回は、最終チェックポイントは無視(False))
trainer.resume_or_load(resume=False)
## 訓練実行
trainer.train()

In [ ]:
# tensorboardを使用して、学習曲線等を可視化
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/output

# 3. 学習済みモデルを用いて推論
- 学習済みモデルとして、上記学習で作成したmodel_final.pthをロード
- 推論なので、アノテーションデータは不要 (ロードしない)


In [ ]:
# コンフィグ (設定)を記述
## コンフィグを初期化
cfg = get_cfg()

# モデルをロード
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# クラス数を指定 (今回は1クラス)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1クラスのみ
# 重みの保存先を指定
cfg.OUTPUT_DIR = "/content/drive/MyDrive/output"
# 重みをロード
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# 閾値を設定 (ここを調整(大きい値を設定)すると、予測確率の低い怪しいオブジェクトは削除される)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
# デバイスモードを設定
cfg.MODEL.DEVICE = "cpu"

# コンフィグを反映した予測器を作成
predictor = DefaultPredictor(cfg)

In [ ]:
# 推論実行
## testフォルダ内のデータを全て読み込み
image_path_list = glob.glob(f"{test_path}/*")

## forループで繰り返し実行
## valフォルダからランダムで3サンプルを抽出
for imagePath in random.sample(image_path_list, 3):
    print("ファイル名 :", imagePath)
    ## 画像の読み込み
    im = cv2.imread(imagePath)
    
    ## 読み込んだ画像をpredictorに入力して推論を実行 (出力 : outputs)
    outputs = predictor(im)

    ## Visualizer(detectron2の画像表示関数)を使用して、推論結果を表示
    v = Visualizer(im[:, :, ::-1],
                   metadata=None, 
                   scale=1.0,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels 元画像をグレースケールに変換するオプション
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

# Q. 予測精度が悪いんじゃないのか？

# A. その通りです。

**原因**
- 学習データが少ない
- 画像のバリエーションが豊富 (背景が画像によってバラバラ)
- アノテーションが雑
- 学習パラメータが未調整

**解決策**
- 学習データ数を増やす
- 背景の変化が少ないデータを使用する
- 正確にアノテーションする
- 学習パラメータを調整する

**すぐにできる解決策**
- 閾値の値を増やすして、誤検出を減らす (<-これはすぐに対応可能)